In [26]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
# from tensorflowcore
# from tensorflow import _tf_uses_legacy_keras
# from keras._tf_keras.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
# import scipy

In [30]:
import sys
print(sys.executable)

C:\Users\avikd\PycharmProjects\pythonProject0\venv\Scripts\python.exe


In [34]:
print(tf.__version__)

2.17.0


In [2]:
# Path where the raw videos are stored on your local machine
video_folder = r'D:\pro_dis\videos\HD_720p'

# Output folder for storing extracted frames (create a new folder for this purpose)
output_folder = r'D:\pro_dis\extracted_frames'
os.makedirs(output_folder, exist_ok=True)

# Function to extract frames from videos without mouth cropping
def extract_frames(video_path, output_dir):
    # Open the video file
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        # Save extracted frame as an image (e.g., JPEG) without cropping
        cv2.imwrite(f"{output_dir}/frame{count}.jpg", image)
        success, image = vidcap.read()
        count += 1

# Process all videos in the folder
for video_name in os.listdir(video_folder):
    if video_name.endswith('.mp4'):  # Ensure you're only processing video files
        video_path = os.path.join(video_folder, video_name)
        output_dir = os.path.join(output_folder, os.path.splitext(video_name)[0])
        os.makedirs(output_dir, exist_ok=True)

        # Extract frames from the video
        print(f"Extracting frames from {video_name}...")
        extract_frames(video_path, output_dir)

print("Frame extraction complete!")

Extracting frames from Bat.mp4...
Extracting frames from Big.mp4...
Extracting frames from Book.mp4...
Extracting frames from Car.mp4...
Extracting frames from Cat.mp4...
Extracting frames from Cup.mp4...
Extracting frames from Dog.mp4...
Extracting frames from Drop.mp4...
Extracting frames from Eat.mp4...
Extracting frames from Fast.mp4...
Extracting frames from Fish.mp4...
Extracting frames from Fun.mp4...
Extracting frames from Go.mp4...
Extracting frames from Hat.mp4...
Extracting frames from Hot.mp4...
Extracting frames from Jump.mp4...
Extracting frames from Kick.mp4...
Extracting frames from Leg.mp4...
Extracting frames from Milk.mp4...
Extracting frames from No.mp4...
Extracting frames from Pen.mp4...
Extracting frames from Pick.mp4...
Extracting frames from Red.mp4...
Extracting frames from Run.mp4...
Extracting frames from Sit.mp4...
Extracting frames from Stop.mp4...
Extracting frames from Sun.mp4...
Extracting frames from Top.mp4...
Extracting frames from Win.mp4...
Extract

In [29]:
# ImageDataGenerator for basic augmentations (horizontal flip, brightness)
datagen = ImageDataGenerator(
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]  # Adjusting brightness (color jittering)
)

# Function to apply noise to an image
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 0.1  # Fixed sigma value (standard deviation) for Gaussian noise
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy_image = np.clip(image + gauss * 255, 0, 255).astype(np.uint8)
    return noisy_image

# Function to augment and save augmented frames
def augment_and_save_frame(image_path, output_dir, count):
    img = cv2.imread(image_path)
    img = img_to_array(img)

    # Apply horizontal flipping and brightness adjustments
    img = datagen.random_transform(img)

    # Add noise to 30% of the frames
    if random.random() < 0.3:
        img = add_noise(img)

    # Convert back to image format and save augmented frame with consistent naming
    aug_img = array_to_img(img)
    aug_img.save(f"{output_dir}/frame{count}.jpg")  # Augmented frame saved with sequential naming

# Function to randomly shift frames temporally
def temporal_shift(frames):
    shift_amount = random.randint(-2, 2)  # Shift between -2 to +2 frames
    return np.roll(frames, shift_amount, axis=0)

# Function to augment frames with all techniques and keep both original and augmented frames
def augment_frames_with_originals(frame_dir, output_dir):
    # List all frame files in the directory
    frames = sorted(os.listdir(frame_dir))

    # Randomly remove or duplicate 10% of frames to simulate frame corruption/duplication
    num_modify = int(0.1 * len(frames))
    modify_indices = random.sample(range(len(frames)), num_modify)

    # Read all frames into a list for easier manipulation
    all_frames = [cv2.imread(os.path.join(frame_dir, frame)) for frame in frames]

    # Apply temporal shifting to the entire sequence
    shifted_frames = temporal_shift(np.array(all_frames))

    # Save original frames to output directory
    for count, frame_name in enumerate(frames):
        frame_path = os.path.join(frame_dir, frame_name)
        original_frame_path = os.path.join(output_dir, f"frame{count}.jpg")
        os.system(f"copy \"{frame_path}\" \"{original_frame_path}\"")  # Use copy for Windows

        # If the frame is selected for modification (removal or duplication)
        if count in modify_indices:
            if random.random() < 0.5:  # Randomly decide whether to remove or duplicate
                # Duplicate frame (copy it again)
                aug_img_path = os.path.join(output_dir, f"frame{count + len(frames)}.jpg")
                os.system(f"copy \"{original_frame_path}\" \"{aug_img_path}\"")  # Duplicate the frame
            else:
                # Skip the frame to simulate removal
                continue

        # Apply augmentation to the frame and save it
        augment_and_save_frame(frame_path, output_dir, count + len(frames))  # Augmented frames start after originals

# Path where the raw videos are stored on your local machine
extracted_frames_folder = r'D:\pro_dis\extracted_frames'  # Path where the extracted frames are stored

# Output folder for organizing both original and augmented frames
organized_frames_folder = r'D:\pro_dis\organized_frames'  # Output folder for augmented frames
os.makedirs(organized_frames_folder, exist_ok=True)

# Apply augmentation and store both original and augmented frames in an organized folder
for word_dir in os.listdir(extracted_frames_folder):
    word_frame_dir = os.path.join(extracted_frames_folder, word_dir)
    output_word_dir = os.path.join(organized_frames_folder, word_dir)  # New output directory
    os.makedirs(output_word_dir, exist_ok=True)  # Create output directory for each word
    if os.path.isdir(word_frame_dir):
        print(f"Organizing frames for {word_dir} with full augmentation...")
        augment_frames_with_originals(word_frame_dir, output_word_dir)

print("Frames organized for deep learning analysis with full augmentation!")

Organizing frames for Bat with full augmentation...


ModuleNotFoundError: No module named 'PIL'